In [3]:
#Headers for the analysis
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option("mode.chained_assignment", None)
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.multioutput import MultiOutputRegressor


In [4]:
# Fixed values for the device, not going to change based on our requirement

freq_dict = {}
freq_dict['gFreq'] = [177000000,266000000,350000000,420000000,480000000,600000000]
freq_dict['mFreq'] = [165000000,206000000,275000000,413000000,543000000,633000000,728000000,825000000]#,933000000]
freq_dict['sFreq'] = [200000,300000,400000,500000,600000,700000,800000,900000,1000000,1100000,1200000,1300000,1400000]
freq_dict['bFreq'] = [200000,300000,400000,500000,600000,700000,800000,900000,1000000,1100000,1200000,1300000,1400000,1500000,1600000,1700000,1800000,1900000,2000000]
cost_list =['cCost','mCost','gCost']

In [8]:
## Function to extract the files and send a dataframe

def extractDatafromFile(fileName='./UtilTestFinal',padding=0):
    new_data = pd.read_csv(fileName)
    #### This part needs to be 
    if padding == 0:  # if there is not 
        new_data['cCost'] =  new_data['bUtil']*new_data['bFreq']/2000000
        new_data['mCost'] = new_data['mUtil']*new_data['mFreq']/825000000
        new_data['gCost'] = 0
        new_data['gFreq'] = 600000000

    return new_data

#extractDatafromFile()

In [ ]:
## Function to shift the data, enabling testing and trainig

def createRequiredDataSet()